In [2]:
from trackml.dataset import load_event, load_dataset
from trackml.score import score_event
from sklearn import preprocessing
import pandas as pd
import numpy as np
import glob, hdbscan

import operator

from tqdm import tqdm

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from scipy.sparse.csgraph import connected_components

from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
%matplotlib inline

In [5]:
scl = preprocessing.StandardScaler()

#https://www.kaggle.com/mikhailhushchyn/dbscan-benchmark
#https://www.kaggle.com/mikhailhushchyn/hough-transform
def norm_points(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2 + y**2 + z**2)
    df['x2'] = x/r
    df['y2'] = y/r
    df['z2'] = z / np.sqrt(x**2 + y**2)
        
    return df

In [3]:
detectors = pd.read_csv('../data/detectors.csv')
# extract phi and theta from a direciton vector
def phi_theta(dx,dy,dz) :
    dr  = np.sqrt(dx*dx+dy*dy)
    phi = np.arctan2(dy,dx)
    theta = np.arctan2(dr,dz)
    return phi, theta

# function to extract the rotation matrix (and its inverse) from module dataframe
def extract_rotation_matrix(module) :
    rot_matrix = np.matrix( [[ module.rot_xu.data[0], module.rot_xv.data[0], module.rot_xw.data[0]],
                            [  module.rot_yu.data[0], module.rot_yv.data[0], module.rot_yw.data[0]],
                            [  module.rot_zu.data[0], module.rot_zv.data[0], module.rot_zw.data[0]]])
    return rot_matrix, np.linalg.inv(rot_matrix)

def extract_rotation_matrix1(module) :
    rot_matrix = np.matrix( [[ module.rot_xu.data[0], module.rot_xv.data[0], module.rot_xw.data[0]],
                            [  module.rot_yu.data[0], module.rot_yv.data[0], module.rot_yw.data[0]],
                            [  module.rot_zu.data[0], module.rot_zv.data[0], module.rot_zw.data[0]]])
    return rot_matrix, np.linalg.inv(rot_matrix)

def add_p_phi_theta_truth(h, truth):
    truth = truth.apply(abs)
    truth['tpr'] = np.sqrt(truth.tpx ** 2 + truth.tpy ** 2)
    truth['p_phi_truth'] = np.arctan2(truth.tpy, truth.tpx)
    truth['p_theta_truth'] = np.arctan2(truth.tpr, truth.tpz)
    return h.merge(truth[['hit_id', 'p_phi_truth', 'p_theta_truth']],
                   on='hit_id', how='left')

def add_cells(h, cells):
    h_clusters = cells.groupby('hit_id') \
                      .agg({'ch0': ['min', 'max'],
                            'ch1': ['min', 'max']}) \
                      .reset_index()
    h_clusters.columns = ['_'.join(column).strip('_')
                          for column in h_clusters.columns.values]
    return h.merge(h_clusters, on='hit_id', how='left')

def add_detectors(h, detectors):
    return h.merge(detectors,
                   on=['volume_id', 'layer_id', 'module_id'], how='left')

def add_p_phi_theta(h):
    h['cluster_size_u_max'] = h.ch0_max - h.ch0_min + 1
    h['cluster_size_u_min'] = h.cluster_size_u_max - 2
    h.loc[h.cluster_size_u_min < 0, 'cluster_size_u_min'] = 0

    h['cluster_size_v_max'] = h.ch1_max - h.ch1_min + 1
    h['cluster_size_v_min'] = h.cluster_size_v_max - 2
    h.loc[h.cluster_size_v_min < 0, 'cluster_size_v_min'] = 0

    h['pu_max'] = h.cluster_size_u_max * h.pitch_u
    h['pu_min'] = h.cluster_size_u_min * h.pitch_u

    h['pv_max'] = h.cluster_size_v_max * h.pitch_v
    h['pv_min'] = h.cluster_size_v_min * h.pitch_v

    h['pw'] = 2 * h.module_t

    h['angle_u_max'] = np.arctan2(h.pu_max, h.pw)
    h['angle_u_min'] = np.arctan2(h.pu_min, h.pw)
    h['angle_u_avg'] = 0.5 * (h.angle_u_max + h.angle_u_min)
    h['pu'] = h.pw * np.tan(h.angle_u_avg)

    h['angle_v_max'] = np.arctan2(h.pv_max, h.pw)
    h['angle_v_min'] = np.arctan2(h.pv_min, h.pw)
    h['angle_v_avg'] = 0.5 * (h.angle_v_max + h.angle_v_min)
    h['pv'] = h.pw * np.tan(h.angle_v_avg)

    h['px'] = abs(h.rot_xu * h.pu + h.rot_xv * h.pv + h.rot_xw * h.pw)
    h['py'] = abs(h.rot_yu * h.pu + h.rot_yv * h.pv + h.rot_yw * h.pw)
    h['pz'] = abs(h.rot_zu * h.pu + h.rot_zv * h.pv + h.rot_zw * h.pw)
    h['pr'] = np.sqrt(h.px ** 2 + h.py ** 2)
    h['p_phi'] = np.arctan2(h.py, h.px)
    h['p_theta'] = np.arctan2(h.pr, h.pz)


In [ ]:
# noise sort by r1, 2 and  2 places, kmeans for all track_count > 15 and 50, NearestNeighbor for <= 15
# using tan_dip
# using phi only for NearestNeighbor
from time import sleep
scores = []
B = 0.002036374942542289
for e in train:
    hits, cells, truth, particles = load_event(e, parts=['hits', 'cells', 'truth', 'particles'])
    hits['event_id'] = int(e[-9:])
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    hits = hits.fillna(0)
#     hits = hits[hits.particle_id != 0]
    hits = norm_points(hits)
    for m in ['braycurtis']: #Tuning/Grid Search
        print(m)
        try:
#             np.random.seed(123) # for reproducability
#             particles_list = list(set(hits.particle_id.values))
#             n_particles = np.random.choice(particles_list, 100)
#             hits = hits[hits.particle_id.isin(n_particles)]
            hits['phi'] = np.degrees(np.arctan2(hits['y2'], hits['x2']))
            hits['phi_p'] = np.degrees(np.arctan2(hits['py'], hits['px']))
            x2 = hits.x2.values
            y2 = hits.y2.values
            z2 = hits.z2.values
            hits['rho'] = np.sqrt(x2**2 + y2**2 + z2**2)
            hits['r'] = np.sqrt(x2**2 + y2**2)
            hits['phi'] = np.degrees(np.arctan2(hits['y2'], hits['x2']))
            hits['theta'] = np.degrees(np.arctan2(hits['r'], hits['z2']))
            phi = hits['phi'].values
            rho = hits['rho'].values
            hits['tan_dip'] = phi/z2
            dbscan = hdbscan.HDBSCAN(min_samples=1, min_cluster_size=7, cluster_selection_method='leaf', prediction_data=False, metric=m)
            labels= dbscan.fit_predict(scl.fit_transform(hits[['z2', 'phi', 'rho', 'r', 'theta', 'tan_dip']].values))
            
            hits['track_id'] = labels
            
            hits['track_id'] = hits['track_id'] + 1
            s = hits['track_id'].value_counts()
            hits['track_count'] = hits['track_id'].map(lambda x: s[x])
            
            
            # select 10 particles at random
            
#             hits = hits[(hits.particle_id == 22525763437723648) | (hits.particle_id == 297237712845406208) |
#            (hits.particle_id == 418835796137607168) | (hits.particle_id == 108087696726949888) 
#            | (hits.particle_id == 968286151951515648)]
            tracks_list = list(set(hits.track_id.values))
            total_noise_rows = 0
            hits_noise = pd.DataFrame()
            
            total_noise_rows_15 = 0
            hits_noise_15 = pd.DataFrame()
            
            total_noise_rows_50 = 0
            hits_noise_50 = pd.DataFrame()
            
            for track in tracks_list:
                hits1 = hits[hits.track_id == track]
                x2 = hits1.x2.values
                y2 = hits1.y2.values
                z2 = hits1.z2.values
                
                if hits1.track_count.data[0] > 50:
                    hits1['r1'] = np.round(np.sqrt(x2**2 + y2**2), 2)
                    n = hits1.r1.nunique()
                    kmeans = KMeans(n_clusters=n)
    
                    z = hits1[['phi', 'r']].values

                    
                    X = cdist(z,z, 'cityblock').astype(np.float64)

                    kmeans.fit(X)
                    y_kmeans = kmeans.predict(X)
                    hits1['y_kmeans'] = y_kmeans
                #     print(hits1)
                    for track2 in range(n):
#                         sleep(0.1)
#                         print('track2 = {}'.format(track2))
                        x = hits1[hits1.y_kmeans == track2].phi.values
                        y = hits1[hits1.y_kmeans == track2].rho.values
#                         print(x.shape)
#                         print(y.shape)
                        if x.shape[0] == 1:
                            B1 = 0.002036374942542289
                            hits1.loc[hits1.y_kmeans == track2, 'myp'] = 0
                            
                        elif x.shape[0] == 2:
                            A = [x[0], y[0]]
                            B = [x[1], y[1]]
                            d = np.sqrt(((x[0]-x[1])**2) + ((y[0]-y[1])**2))
#                             print(d)
                            B1 = 0.002036374942542289
                            p = 0.3 * B1 * d * 2
                            p = d
                            if np.isnan(p):
                                p = 0
                                print('d: {}'.format(d))
                                print(x[0], x[1], y[0], y[1])
                            hits1.loc[hits1.y_kmeans == track2, 'myp'] = p

                        elif x.shape[0] == 3:
                            points_utm = np.zeros((3,2))
                            points_utm[0, 0] = x[0]
                            points_utm[0, 1] = y[0]


                            points_utm[1, 0] = x[1]
                            points_utm[1, 1] = y[1]


                            points_utm[2, 0] = x[2]
                            points_utm[2, 1] = y[2]

                            curvature = get_curvature1(points_utm)
                            R = 1/curvature
                            B1 = 0.002036374942542289
                            p = 0.3 * B1 * R * 2
                            p = R
                            if np.isnan(p):
                                p = 0
                                print('R 3pts: {}'.format(R))
                                print(x[0], x[1], y[0], y[1], x[2], y[2])
                            hits1.loc[hits1.y_kmeans == track2, 'myp'] = p
                        elif x.shape[0] > 3:
                            x = hits1[hits1.y_kmeans == track2].phi.values
                            y = hits1[hits1.y_kmeans == track2].rho.values
                            comp_curv = ComputeCurvature()
                            curvature = comp_curv.fit(x, y)
                            R = 1/curvature
                            B1 = 0.002036374942542289
                            p = 0.3 * B1 * R * 2
                            p = R
                            if np.isnan(p):
                                p = 0
                                print('R: {}'.format(R))
                            hits1.loc[hits1.y_kmeans == track2, 'myp'] = p
                    hits.loc[hits.track_id == track, 'myp'] = hits1.myp.values
                    
                    total_noise_rows += len(hits1)
                    hits_noise = hits_noise.append(hits1)
                    
                    total_noise_rows_50 += len(hits1)
                    hits_noise_50 = hits_noise_50.append(hits1)
                        
                
                else:
                    z = hits1[['phi']].values
                    X = z
                    nbrs = NearestNeighbors(n_neighbors=3, algorithm='auto').fit(X)
                    n, ii_labels = connected_components(nbrs.kneighbors_graph(X).toarray())
                    hits1['track'] = ii_labels
    #                 print('------')

                    for track2 in list(set(ii_labels)):
                        x = hits1[hits1.track == track2].phi.values
                        y = hits1[hits1.track == track2].rho.values

                        x_0, y_0, z_0 = x[0], y[0], z[0]
                        x_1, y_1, z_1 = x[1], y[1], z[1]

                        if x.shape[0] == 1 :
                            hits1.loc[hits1.track == track2, 'myp'] = 0.3 * B * np.sqrt(x**2 + y**2) * 2
                            continue
                        comp_curv = ComputeCurvature()
                        curvature = comp_curv.fit(x, y)
                        R = 1/curvature
                        B1 = 0.002036374942542289
                        p = 0.3 * B1 * R * 2
                        p = R
                        hits1.loc[hits1.track == track2, 'myp'] = p
                    hits.loc[hits.track_id == track, 'myp'] = hits1.myp.values
#                     total_noise_rows += len(hits1)
#                     hits_noise = hits_noise.append(hits1)
#                     continue
                

            p = hits['myp'].values
            
            hits['x3'] = hits['x2'].values + p
            hits['y3'] = hits['y2'].values + p
            hits['z3'] = hits['z2'].values + p
            

            dbscan = hdbscan.HDBSCAN(min_samples=1, min_cluster_size=7, cluster_selection_method='leaf', prediction_data=False, metric=m)

#             labels= dbscan.fit_predict(scl.fit_transform(hits[['x3', 'y3', 'z3']].values))+1
            labels= dbscan.fit_predict(scl.fit_transform(hits[['z2', 'phi', 'rho', 'r', 'theta', 'tan_dip', 'myp']].values))
            hits['track_id'] = labels
            
            score = score_event(hits, hits[['event_id','hit_id','track_id']])
            print(m, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, m])
            
            labels= dbscan.fit_predict(scl.fit_transform(hits[['z3', 'phi', 'rho', 'r', 'theta', 'tan_dip']].values))
            hits['track_id'] = labels
            
            score = score_event(hits, hits[['event_id','hit_id','track_id']])
            print(m, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, m])
            
            labels= dbscan.fit_predict(scl.fit_transform(hits[['x3', 'y3', 'z3', 'phi', 'rho', 'r', 'theta', 'tan_dip']].values))
            hits['track_id'] = labels
            
            score = score_event(hits, hits[['event_id','hit_id','track_id']])
            print(m, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, m])
            
            labels= dbscan.fit_predict(scl.fit_transform(hits[['z2', 'phi', 'rho', 'r', 'theta', 'tan_dip']].values))
            hits['track_id'] = labels
            
            score = score_event(hits, hits[['event_id','hit_id','track_id']])
            print(m, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)
            scores.append([score, m])
            
            noise_hit_ids = hits_noise.hit_id.values
            hits_processed = hits[~hits.hit_id.isin(noise_hit_ids)]
            score = score_event(hits_processed, hits_processed[['event_id','hit_id','track_id']])
            print(m, len(hits_processed['particle_id'].unique()), len(hits_processed['track_id'].unique()), score)
            
            score = score_event(hits_noise, hits_noise[['event_id','hit_id','track_id']])
            print(m, len(hits_noise['particle_id'].unique()), len(hits_noise['track_id'].unique()), score)
            
#             noise_hit_ids_15 = hits_noise_15.hit_id.values
#             hits_processed_15 = hits[hits.hit_id.isin(noise_hit_ids_15)]
#             score = score_event(hits_processed_15, hits_processed_15[['event_id','hit_id','track_id']])
#             print(m, len(hits_processed_15['particle_id'].unique()), len(hits_processed_15['track_id'].unique()), score)
            
            noise_hit_ids_50 = hits_noise_50.hit_id.values
            hits_processed_50 = hits[hits.hit_id.isin(noise_hit_ids_50)]
            score = score_event(hits_processed_50, hits_processed_50[['event_id','hit_id','track_id']])
            print(m, len(hits_processed_50['particle_id'].unique()), len(hits_processed_50['track_id'].unique()), score)
            
            print(hits.shape)
            print(total_noise_rows)
            print(total_noise_rows/hits.shape[0])
            
        except e:
            print("ERROR:", e)
#         break #Remove to test all
    break #Remove to test more samples
print(sorted(scores, reverse=True))

In [33]:
event_id = 'event000001038'
# "All methods either take or return pandas.DataFrame objects"
hits, cells, particles, truth = load_event('../data/train/'+event_id)

norm_points(hits)
hits = add_cells(hits, cells)
hits = add_detectors(hits, detectors)
add_p_phi_theta(hits)

# hits['phi_p'] = np.degrees(np.arctan2(hits['py'], hits['px']))
x2 = hits.x2.values
y2 = hits.y2.values
z2 = hits.z2.values

x = hits.x.values
y = hits.y.values
z = hits.z.values
hits['rho'] = np.sqrt(x**2 + y**2 + z**2)
hits['r'] = np.sqrt(x**2 + y**2)
hits['r2'] = np.sqrt(hits['x2']**2 + hits['y2']**2)
hits['phi'] = np.degrees(np.arctan2(hits['y2'], hits['x2']))
hits['theta'] = np.degrees(np.arctan2(hits['r'], hits['z']))
phi = hits['phi'].values
rho = hits['rho'].values
hits['tan_dip'] = phi/z2
m = 'manhattan'
dbscan = hdbscan.HDBSCAN(min_samples=1, min_cluster_size=7, cluster_selection_method='leaf', prediction_data=False, metric=m)
# labels= dbscan.fit_predict(scl.fit_transform(hits[['z2', 'phi', 'rho', 'r', 'theta', 'tan_dip']].values))
hits['x3'] = hits['x2'] + (hits['z']/hits['r'])
hits['y3'] = hits['y2'] + (hits['z']/hits['r'])
hits['z3'] = hits['z2'] + (hits['z']/hits['r'])
# labels= dbscan.fit_predict(scl.fit_transform(hits[['x3', 'y3', 'z3']].values))
labels= dbscan.fit_predict(scl.fit_transform(hits[['z2', 'phi', 'r2', 'theta']].values))
hits['track_id'] = labels+1
hits['event_id'] = 1038
score = score_event(truth, hits[['event_id','hit_id','track_id']])
print(m, len(truth['particle_id'].unique()), len(hits['track_id'].unique()), score)

manhattan 9820 7164 0.26396742892358765


In [15]:
hits.head()

,hit_id,x,y,z,volume_id,layer_id,module_id,x2,y2,z2,ch0_min,ch0_max,ch1_min,ch1_max,cx,cy,cz,rot_xu,rot_xv,rot_xw,rot_yu,rot_yv,rot_yw,rot_zu,rot_zv,rot_zw,module_t,module_minhu,module_maxhu,module_hv,pitch_u,pitch_v,cluster_size_u_max,cluster_size_u_min,cluster_size_v_max,cluster_size_v_min,pu_max,pu_min,pv_max,pv_min,pw,angle_u_max,angle_u_min,angle_u_avg,pu,angle_v_max,angle_v_min,angle_v_avg,pv,px,py,pz,pr,p_phi,p_theta,rho,r,phi,theta,tan_dip,x3,y3,z3,track_id,event_id
0,1,-57.259499,-12.20520,-1502.5,7,2,1,-0.038081,-0.008117,-25.663643,321,321,498,498,-65.7965,-5.1783,-1502.5,0.078459,-0.996917,0.0,-0.996917,-0.078459,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625,1,0,1,0,0.05,0.0,0.05625,0.0,0.3,0.165149,0.0,0.082574,0.024829,0.185348,0.0,0.092674,0.027882,0.025848,0.02694,0.3,0.037335,0.806077,0.123812,25.663673,0.038936,-167.967148,179.913055,6.544946,-0.000746,0.029218,-25.626308,1606,1038
1,2,-69.977097,-13.35650,-1502.5,7,2,1,-0.046522,-0.008880,-21.090572,324,324,725,725,-65.7965,-5.1783,-1502.5,0.078459,-0.996917,0.0,-0.996917,-0.078459,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625,1,0,1,0,0.05,0.0,0.05625,0.0,0.3,0.165149,0.0,0.082574,0.024829,0.185348,0.0,0.092674,0.027882,0.025848,0.02694,0.3,0.037335,0.806077,0.123812,21.090626,0.047361,-169.193939,179.871323,8.022255,-0.009187,0.028455,-21.053238,2452,1038
2,3,-66.729301,-3.72199,-1502.5,7,2,1,-0.044368,-0.002475,-22.481401,137,137,654,654,-65.7965,-5.1783,-1502.5,0.078459,-0.996917,0.0,-0.996917,-0.078459,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625,1,0,1,0,0.05,0.0,0.05625,0.0,0.3,0.165149,0.0,0.082574,0.024829,0.185348,0.0,0.092674,0.027882,0.025848,0.02694,0.3,0.037335,0.806077,0.123812,22.481445,0.044437,-176.807480,179.886734,7.864611,-0.007034,0.034860,-22.444067,2361,1038
3,4,-64.853401,-12.50190,-1502.5,7,2,1,-0.043122,-0.008313,-22.748808,315,315,633,633,-65.7965,-5.1783,-1502.5,0.078459,-0.996917,0.0,-0.996917,-0.078459,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625,1,0,1,0,0.05,0.0,0.05625,0.0,0.3,0.165149,0.0,0.082574,0.024829,0.185348,0.0,0.092674,0.027882,0.025848,0.02694,0.3,0.037335,0.806077,0.123812,22.748850,0.043916,-169.088821,179.889389,7.432865,-0.005787,0.029022,-22.711473,2458,1038
4,5,-62.312302,-8.74091,-1502.5,7,2,1,-0.041436,-0.005812,-23.878624,244,244,582,583,-65.7965,-5.1783,-1502.5,0.078459,-0.996917,0.0,-0.996917,-0.078459,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625,1,0,2,0,0.05,0.0,0.11250,0.0,0.3,0.165149,0.0,0.082574,0.024829,0.358771,0.0,0.179385,0.054400,0.052285,0.02902,0.3,0.059799,0.506710,0.196750,23.878660,0.041842,-172.014877,179.899597,7.203718,0.018362,0.053986,-23.818825,2435,1038
